# Description

This notebook generates more end-user-friendly Excel files of some of the data generated in PhenoPLIER, such as the LV-gene matrix and LV-pathways.

# Modules loading

In [1]:
import pandas as pd
import openpyxl
from openpyxl.utils import get_column_letter

import conf
from utils import get_git_repository_path

# Settings

In [2]:
DELIVERABLES_BASE_DIR = get_git_repository_path() / "data"
display(DELIVERABLES_BASE_DIR)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/data')

In [3]:
OUTPUT_DIR = DELIVERABLES_BASE_DIR / "multiplier"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
display(OUTPUT_DIR)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/data/multiplier')

# Load data

## MultiPLIER summary

In [4]:
multiplier_model_summary = pd.read_pickle(conf.MULTIPLIER["MODEL_SUMMARY_FILE"])

In [5]:
multiplier_model_summary.shape

(2157, 5)

In [6]:
multiplier_model_summary.head()

,pathway,LV index,AUC,p-value,FDR
1,KEGG_LYSINE_DEGRADATION,1,0.388059,0.866078,0.956005
2,REACTOME_MRNA_SPLICING,1,0.733057,0.000048,0.000582
3,MIPS_NOP56P_ASSOCIATED_PRE_RRNA_COMPLEX,1,0.680555,0.001628,0.011366
4,KEGG_DNA_REPLICATION,1,0.549473,0.312155,0.539951
5,PID_MYC_ACTIVPATHWAY,1,0.639303,0.021702,0.083739


## Get pathway-aligned LVs

In [7]:
well_aligned_lvs = multiplier_model_summary[multiplier_model_summary["FDR"] < 0.05]

display(well_aligned_lvs.shape)
display(well_aligned_lvs.head())

[2021-08-20 11:58:20,586 - numexpr.utils] INFO: NumExpr defaulting to 4 threads.


(463, 5)

,pathway,LV index,AUC,p-value,FDR
2,REACTOME_MRNA_SPLICING,1,0.733057,4.772691e-05,5.816211e-04
3,MIPS_NOP56P_ASSOCIATED_PRE_RRNA_COMPLEX,1,0.680555,1.628217e-03,1.136590e-02
8,REACTOME_MITOTIC_G1_G1_S_PHASES,1,0.686170,2.517619e-04,2.392292e-03
9,IRIS_Monocyte-Day0,2,0.890036,4.315812e-25,1.329887e-22
10,DMAP_MONO2,2,0.904676,1.313970e-16,1.574574e-14


In [8]:
well_aligned_lv_codes = set([f"LV{lvi}" for lvi in well_aligned_lvs["LV index"]])

In [9]:
len(well_aligned_lv_codes)

199

In [10]:
list(well_aligned_lv_codes)[:5]

['LV915', 'LV123', 'LV58', 'LV881', 'LV848']

## MultiPLIER Z (gene loadings)

In [11]:
multiplier_z = pd.read_pickle(conf.MULTIPLIER["MODEL_Z_MATRIX_FILE"])

In [12]:
multiplier_z.shape

(6750, 987)

In [13]:
multiplier_z.head()

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,...,LV978,LV979,LV980,LV981,LV982,LV983,LV984,LV985,LV986,LV987
GAS6,0.000000,0.0,0.039438,0.0,0.050476,0.000000,0.0,0.000000,0.590949,0.000000,...,0.050125,0.00000,0.033407,0.000000,0.000000,0.005963,0.347362,0.0,0.000000,0.000000
MMP14,0.000000,0.0,0.000000,0.0,0.070072,0.000000,0.0,0.004904,1.720179,2.423595,...,0.000000,0.00000,0.001007,0.000000,0.035747,0.000000,0.000000,0.0,0.014978,0.000000
DSP,0.000000,0.0,0.000000,0.0,0.000000,0.041697,0.0,0.005718,0.000000,0.000000,...,0.020853,0.00000,0.000000,0.000000,0.000000,0.005774,0.000000,0.0,0.000000,0.416405
MARCKSL1,0.305212,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.161843,0.149471,...,0.027134,0.05272,0.000000,0.030189,0.060884,0.000000,0.000000,0.0,0.000000,0.448480
SPARC,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.014014,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.067779,0.0,0.122417,0.062665


# Create LV-pathway dataframe

In [14]:
lv_pathway_df = multiplier_model_summary[
    ["LV index", "pathway", "AUC", "p-value", "FDR"]
]

In [15]:
lv_pathway_df["LV index"] = lv_pathway_df["LV index"].astype(int)

In [16]:
lv_pathway_df = lv_pathway_df.sort_values(["LV index", "FDR"])

In [17]:
lv_pathway_df["LV index"] = lv_pathway_df["LV index"].apply(lambda x: f"LV{x}")

In [18]:
lv_pathway_df.shape

(2157, 5)

In [19]:
lv_pathway_df = lv_pathway_df.rename(
    columns={"LV index": "LV identifier", "pathway": "Pathway"}
)

In [20]:
lv_pathway_df.head()

,LV identifier,Pathway,AUC,p-value,FDR
2,LV1,REACTOME_MRNA_SPLICING,0.733057,0.000048,0.000582
8,LV1,REACTOME_MITOTIC_G1_G1_S_PHASES,0.686170,0.000252,0.002392
3,LV1,MIPS_NOP56P_ASSOCIATED_PRE_RRNA_COMPLEX,0.680555,0.001628,0.011366
5,LV1,PID_MYC_ACTIVPATHWAY,0.639303,0.021702,0.083739
6,LV1,REACTOME_METABOLISM_OF_NUCLEOTIDES,0.596122,0.106494,0.266826


In [21]:
lv_pathway_df.tail()

,LV identifier,Pathway,AUC,p-value,FDR
2156,LV987,PID_LKB1_PATHWAY,0.707349,0.013615,0.059448
2153,LV987,REACTOME_PRE_NOTCH_EXPRESSION_AND_PROCESSING,0.623659,0.122716,0.293872
2155,LV987,PID_E2F_PATHWAY,0.582317,0.137210,0.316974
2157,LV987,REACTOME_INTERACTIONS_OF_VPR_WITH_HOST_CELLULA...,0.524430,0.415924,0.643117
2154,LV987,KEGG_LYSINE_DEGRADATION,0.376037,0.877868,0.960271


## Save

In [22]:
output_file = OUTPUT_DIR / "lv-pathways.xlsx"
display(output_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/data/multiplier/lv-pathways.xlsx')

In [23]:
lv_pathway_df.to_excel(output_file, index=False)

In [24]:
# adjust column widths
wb = openpyxl.load_workbook(filename=output_file)
worksheet = wb.active

for col in worksheet.columns:
    max_length = 0
    column = get_column_letter(col[0].column)  # Get the column name
    for cell in col:
        if cell.coordinate in worksheet.merged_cells:  # not check merge_cells
            continue

        try:  # Necessary to avoid error on empty cells
            if len(str(cell.value)) > max_length:
                max_length = len(str(cell.value))
        except:
            pass
    adjusted_width = (max_length + 2) * 1.05
    worksheet.column_dimensions[column].width = adjusted_width

wb.save(output_file)

# Create LV-gene dataframe

In [25]:
df = (
    multiplier_z.unstack()
    .to_frame()
    .reset_index()
    .rename(columns={0: "Weight", "level_0": "LV identifier", "level_1": "Gene symbol"})
)

In [26]:
df = df.assign(lv_index=df["LV identifier"].apply(lambda x: int(x[2:])))

In [27]:
df = df.sort_values(["lv_index", "Weight"], ascending=[True, False]).drop(
    columns=["lv_index"]
)

In [28]:
df.shape

(6662250, 3)

In [29]:
df.head()

,LV identifier,Gene symbol,Weight
4370,LV1,POLD1,2.989508
3303,LV1,TRIM28,2.967235
6581,LV1,TOMM40,2.941369
4915,LV1,PKMYT1,2.810091
6549,LV1,CDT1,2.776054


In [30]:
df.tail()

,LV identifier,Gene symbol,Weight
6662237,LV987,LDHD,0.0
6662240,LV987,LDHB,0.0
6662242,LV987,ACAP2,0.0
6662243,LV987,ACAP3,0.0
6662245,LV987,CFL1,0.0


## Save as TSV

In [31]:
# output_file = OUTPUT_DIR / "lv-gene_weights.tsv.gz"
# display(output_file)

In [32]:
# df.to_csv(output_file, sep="\t", index=False)

## Save as Excel

In [33]:
output_file = OUTPUT_DIR / "lv-gene_weights.xlsx"
display(output_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/data/multiplier/lv-gene_weights.xlsx')

In [34]:
with pd.ExcelWriter(output_file) as writer:
    for lv_id in df["LV identifier"].unique():
        print(lv_id, end=", ", flush=True)

        lv_data = df[df["LV identifier"] == lv_id].drop(columns=["LV identifier"])
        lv_data = lv_data.sort_values("Weight", ascending=False)

        lv_data.to_excel(writer, index=False, sheet_name=lv_id)

LV1, LV2, LV3, LV4, LV5, LV6, LV7, LV8, LV9, LV10, LV11, LV12, LV13, LV14, LV15, LV16, LV17, LV18, LV19, LV20, LV21, LV22, LV23, LV24, LV25, LV26, LV27, LV28, LV29, LV30, LV31, LV32, LV33, LV34, LV35, LV36, LV37, LV38, LV39, LV40, LV41, LV42, LV43, LV44, LV45, LV46, LV47, LV48, LV49, LV50, LV51, LV52, LV53, LV54, LV55, LV56, LV57, LV58, LV59, LV60, LV61, LV62, LV63, LV64, LV65, LV66, LV67, LV68, LV69, LV70, LV71, LV72, LV73, LV74, LV75, LV76, LV77, LV78, LV79, LV80, LV81, LV82, LV83, LV84, LV85, LV86, LV87, LV88, LV89, LV90, LV91, LV92, LV93, LV94, LV95, LV96, LV97, LV98, LV99, LV100, LV101, LV102, LV103, LV104, LV105, LV106, LV107, LV108, LV109, LV110, LV111, LV112, LV113, LV114, LV115, LV116, LV117, LV118, LV119, LV120, LV121, LV122, LV123, LV124, LV125, LV126, LV127, LV128, LV129, LV130, LV131, LV132, LV133, LV134, LV135, LV136, LV137, LV138, LV139, LV140, LV141, LV142, LV143, LV144, LV145, LV146, LV147, LV148, LV149, LV150, LV151, LV152, LV153, LV154, LV155, LV156, LV157, LV158, LV

In [35]:
# adjust column widths
wb = openpyxl.load_workbook(filename=output_file)

for worksheet in wb.worksheets:
    for col in worksheet.columns:
        max_length = 0
        column = get_column_letter(col[0].column)  # Get the column name
        for cell in col:
            if cell.coordinate in worksheet.merged_cells:  # not check merge_cells
                continue

            try:  # Necessary to avoid error on empty cells
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = (max_length + 2) * 1.05
        worksheet.column_dimensions[column].width = adjusted_width

wb.save(output_file)